<a href="https://colab.research.google.com/github/redinbluesky/the-lm-book/blob/main/04_트랜스포머.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목차
* [Chapter 4_0 개요](#chapter4_0)
* [Chapter 4_1 디코더 블록](#chapter4_1)
* [Chapter 4_2 셀프 어텐션](#chapter4_2)

## Chapter 4_0 개요 <a class="anchor" id="chapter4_0"></a>
1. 트랜스포머 모델은 NLP 분야를 크개 발전시켰다.
    - 넓은 범위의 의존성을 관리하는 데 있어서 RNN의 한계를 극복하고 입력 시퀸스를 병렬로 처리할 수 있다.

2. 트랜스포머 모델에는 세 가지 주요 유형이 있다.
    - 인코더-디코더 트랜스포머: 번역과 같은 시퀸스-투-시퀸스 작업에 사용된다.
    - 인코더 전용 트랜스포머: BERT와 같은 모델로, 주로 이해 작업에 사용된다.
    - 디코더 전용 트랜스포머: GPT 시리즈와 같은 모델로, 주로 생성 작업에 사용된다.

3. 트랜스포머 구조의 핵심 구성 요소는 다음과 같다.
    - 셀프 어텐션 메커니즘: 입력 시퀸스의 다른 위치에 있는 단어들과 얼마나 관련이 있는지 평가한다.
    - 포지셔널 인코딩: 시퀸스 내에서 단어의 순서를 나타낸다.
    - 피드포워드 신경망: 각 위치에서 독립적으로 적용되는 완전 연결 신경망이다.

4. 트랜스포머는 모든 토큰을 동시에 처리한다.
    - 토큰을 병렬 처리함에도 불구하고, 위치 인코딩을 사용해 순차적인 맥락을 유지한다.

5. 디코더 기반 트랜스포머는 아래의 그림처럼 디코더 블록이라는 층을 수직으로 여러 개 쌓아 구성된다.
    - 디코더를 훈련시키려면 입력 시퀸스와 한 토큰씩 앞으로 이동시킨 출력 시퀸스가 필요하다.

        ![Transformer](image/04-00-transformer.png)

## Chapter 4_1 디코더 블록 <a class="anchor" id="chapter4_1"></a>
1. 디코더 블록은 두 개의 하위 층이 있다.
    - 셀프 어텐션과 위치별 다층 퍼셉트론(MLP)이다.

2. 디코더 블록에서 아래와 같은 작업이 이루어진다.
    - 토큰 임베딩을 처리한다.
    - 셀프 어텐션 층은 1부터 L까지 모든 토크 t에 대해 입력 임베딩 벡터 x_t를 세로운 벡터 g_t로 변환한다.
       - L: 입력 길이
    - 셀프 어텟션 다음에는 위치별 MLP가 각 벡터 g_t를 독립적으로 처리한다.
    - 디코더 블록마다 독립적인 파라미터를 가진 MLP가 있다.
    - MLP 하나가 g_t를 입력으로 받아 z_t를 출력한다.
    - MLP가 출력하는 z_t의 개수는 입력 토큰 x_t의 개수와 같다.
    - 다음 출력 벡터 z_t는 다음 디코더 블록의 입력으로 사용된다.
    - 위의 과정이 모든 디코더 블록에서 반복되며, 출력 벡터의 개수는 입력 토큰 x_t의 개수와 같다.
    
        ![Decoder Block](image/04-01-decoder-block.png)

## Chapter 4_2 셀프 어텐션 <a class="anchor" id="chapter4_2"></a>
1. 5개의 토큰을 가진 샘플 ["We","train","a","transformer","model"]과 최대 입력 길이가 4인 디코더가 있다고 가정
    - 각 토큰은 6차원 임베딩 벡터로 표현된다고 가정
    - 각 디코더 블록에서 셀프 어텐션은 훈련 가능한 세 개의 파라미터 행렬 W<sup>q</sup>, W<sup>k</sup>, W<sup>v</sup>를 사용
        - q: 쿼리(query), k: 키(key), v: 값(value)
        - 파라미터 행령의 텐서의 크기가 6 * 6 이라고 가정
    - 입력 임베딩 벡터(4 * 6)와 훈련 가능한 파라미터(6 * 6)와 곱해져 쿼리 행렬 Q, 키 행렬 K, 값 행렬 V(각각 4*6)가 생성된다.

2. "train" 단어를 표현하는 두 번째 토큰 x<sub>2</sub>에 대한 출력 g<sub>2</sub>을 계산하기 위해 셀프 어텐션 층은 6단계를 거친다.

3. 첫 번째 단계
    - 아래와 그림과 같이 쿼리 행렬 Q, 키 행렬 K, 값 행렬 V를 계산한다.
    - 임베딩 벡터 x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>, x<sub>4</sub>에 각각 W<sup>q</sup>, W<sup>k</sup>, W<sup>v</sup>를 곱한다.
    - 행렬 곱셈의 결과로 쿼리 행렬 Q, 키 행렬 K, 값 행렬 V가 생성된다.
    - 이 행렬은 각각 4행 6열의 크기를 가진다.
    - 각각의 입력 임베딩 K<sub>t</sub>는 쿼리 벡터 Q<sub>t</sub>와 값 벡터 V<sub>t</sub>, 키 벡터 K<sub>t</sub>와 매핑된다.
    
        ![Self-Attention](image/04-02-self-attention-1.png)

4. 두 번째 단계
    - 두 번째 토큰 "train"을 처리하는 것을 가정한다.
    - 쿼리 행렬 Q에서 두 번째 행 Q<sub>2</sub>와 키 행렬을 점곱하여 어텐션 점수를 계산한다.
        - Q<sub>2</sub> • K<sub>1</sub> = 4.90
        - Q<sub>2</sub> • K<sub>2</sub> = 17.15
        - Q<sub>2</sub> • K<sub>3</sub> = 9.80
        - Q<sub>2</sub> • K<sub>4</sub> = 12.25
    - 벡터로 나타내면 아래와 같다.
        - 어텐션 점수<sub>2</sub> = [4.90, 17.15, 9.80, 12.25]<sup>T</sup>
    
5. 세 번째 단계
    - 어텐션 점수를 스케일링한다.
    - 어텐션 점수를 키 벡터의 차원 수의 제곱근(√6 ≈ 2.45)으로 나눈다.
    - 스케일된 어텐션 점수<sub>2</sub> =[4.90/2.45, 17.15/2.45, 9.80/2.45, 12.25/2.45]<sup>T</sup>=[2.00, 7.00, 4.00, 5.00]<sup>T</sup>

6. 네 번째 단계
    - 스케일된 어텐션 점수에 코잘 마스크를 적용한다.
        - 코잘 마스크는 미래 토큰에 대한 어텐션 점수를 음의 무한대로 설정한다.
    - 두 번째 입력 위치에 대한 코잘 마스크는 아래와 같다.
        - 코잘 마스크<sub>2</sub> =[0, 0, -∞, -∞]<sup>T</sup>
    - 스케일 조정된 점수에 코잘 마스크를 더해 마스크드 점수를 얻는다.
        - 마스크드 점수<sub>2</sub> = 스케일 조정됨 점수<sub>2</sub> + 코잘 마스크<sub>2</sub> = [2.00, 7.00, -∞, -∞]<sup>T</sup>

7. 다섯 번째 단계
    - 마스크드 점수에 소프트맥스 함수를 적용해 어텐션 가중치를 계산한다.
    - 어텐션 가중치<sub>2</sub> = softmax([2.00, 7.00, -∞, -∞]) 
    - = [e^(2.00)/ (e^(2.00) + e^(7.00)), e^(7.00)/ (e^(2.00) + e^(7.00)), 0.0, 0.0]<sup>T</sup> 
    - = [0.0025, 0.9975, 0.0, 0.0]<sup>T</sup>
    - 어덴션 점수를 키 차원의 제곱근으로 나누면 차원이 커짐에 따라 점곱 결과가 너무 커지는 것을 방지할 수 있다.
    - 큰 점곱과 소프트맥스를 적용하면 그레이디언트가 매우 작아져 학습이 어려워진다.\
    
8. 여섯 번째 단계
    - 어텐션 가중치와 값 벡터 V<sub>1</sub>, V<sub>2</sub>, V<sub>3</sub>, V<sub>4</sub>를 가중합해 출력 벡터 g<sub>2</sub>를 계산한다.
        - g<sub>2</sub> ≈ 0.0025 * V<sub>1</sub> + 0.9975 * V<sub>2</sub> + 0.0 * V<sub>3</sub> + 0.0 * V<sub>4</sub>
    - 위치 2에 대한 디코더의 출력은 위치 1과 위치 2에 있는 입력에만 의존며, 2의 출력에 좀더 큰 가중치를 부여한다.
    - 미래 위치를 참조하지 못하도록 하는 코잘 마스크의 효과이다.
    - 이 예제는 두 번째 토큰이 자기 자신에게 주의를 기울이지만, 문맥에 따라 다른 토큰에 주의를 기울일 수도있다.

9. 벡터 Q, K, V는 다음과 같이 이해할 수 있다.